In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from queue import PriorityQueue 
import cv2
import argparse
import time
import math

In [ ]:
def standardLine(p1, p2):
    # ax+by+d=0
    assert(p1!=p2),"point1 equals to point2, cannot form line"

    tangent_vector = (p2[0]-p1[0], p2[1]-p1[1])
    if (tangent_vector[0]==0):
        normal_vector = (1,0)
    elif (tangent_vector[1]==0): 
        normal_vector = (0,1)
    else:
        normal_vector = (1/(p2[0]-p1[0]), -1/(p2[1]-p1[1]))
    a, b = normal_vector
    norm = np.sqrt(pow(a, 2) + pow(b, 2))
    a, b = a / norm, b / norm 
    d = -(a * p1[0] + b * p1[1])
    return a, b, d


In [ ]:
class Map:
    #mm
    width = 600 
    height = 250 
    occGrid = np.zeros((height+1, width+1))
    robot_radius = 5
    
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal

    @classmethod
    def generateOccGrid(self): 
        
        # boundary
        boundaryLine_1 = standardLine((0,0), (0,250))
        boundaryLine_2 = standardLine((0,250), (600,250))
        boundaryLine_3 = standardLine((600,250), (600,0))
        boundaryLine_4 = standardLine((0,0), (600,0))

        # 
        # triangle
        triangle_1 = standardLine((460, 225), (460, 25))
        triangle_2 = standardLine((460, 225), (510, 125))
        triangle_3 = standardLine((510, 125), (460, 25))


        # upper rectangle
        upperRectangleLine_1 = standardLine((100, 250), (100, 150))
        upperRectangleLine_2 = standardLine((150, 250), (150, 150))
        upperRectangleLine_3 = standardLine((100, 150), (150, 150))

        # upper rectangle
        lowerRectangleLine_1 = standardLine((100, 100), (100, 0))
        lowerRectangleLine_2 = standardLine((150, 100), (150, 0))
        lowerRectangleLine_3 = standardLine((100, 100), (150, 100))

        # hexagon
        edge =  75
        hexagonLine_1 = standardLine((235, 125 + edge/2), (235, 125 - edge/2))
        hexagonLine_2 = standardLine((235, 125 + edge/2), (300, 125 + edge))
        hexagonLine_3 = standardLine((300, 125 + edge), (365, 125 + edge/2))
        hexagonLine_4 = standardLine((365, 125 + edge/2), (365, 125 - edge/2))
        hexagonLine_5 = standardLine((300, 125 - edge), (365, 125 - edge/2))
        hexagonLine_6 = standardLine((235, 125 - edge/2), (300, 125 - edge))

        rows, cols = Map.occGrid.shape 
        for i in range(0, rows):
            for j in range(0, cols): 
                # transform from top-left (0,0) to bottom-left (0,0)
                x = j
                y = rows - 1 - i

                # boundary with clearance
                if ((boundaryLine_1[0] * x + boundaryLine_1[1] * y + boundaryLine_1[2]) <=  Map.robot_radius or \
                    (boundaryLine_2[0] * x + boundaryLine_2[1] * y + boundaryLine_2[2]) >= -Map.robot_radius or \
                    (boundaryLine_3[0] * x + boundaryLine_3[1] * y + boundaryLine_3[2]) >= -Map.robot_radius or \
                    (boundaryLine_4[0] * x + boundaryLine_4[1] * y + boundaryLine_4[2]) <=  Map.robot_radius ): 
                    Map.occGrid[i, j]=2

                    # boundary
                    if ((boundaryLine_1[0] * x + boundaryLine_1[1] * y + boundaryLine_1[2]) <= 0 or \
                        (boundaryLine_2[0] * x + boundaryLine_2[1] * y + boundaryLine_2[2]) >= 0 or \
                        (boundaryLine_3[0] * x + boundaryLine_3[1] * y + boundaryLine_3[2]) >= 0 or \
                        (boundaryLine_4[0] * x + boundaryLine_4[1] * y + boundaryLine_4[2]) <= 0 ): 
                        Map.occGrid[i, j]=1

                # triangle with clearance
                if ((triangle_1[0] * x + triangle_1[1] * y + triangle_1[2]) >= -Map.robot_radius and \
                    (triangle_2[0] * x + triangle_2[1] * y + triangle_2[2]) <=  Map.robot_radius and \
                    (triangle_3[0] * x + triangle_3[1] * y + triangle_3[2]) >= -Map.robot_radius): 
                    Map.occGrid[i, j]=2

                    # triangle
                    if ((triangle_1[0] * x + triangle_1[1] * y + triangle_1[2]) >=0 and \
                        (triangle_2[0] * x + triangle_2[1] * y + triangle_2[2]) <=0 and \
                        (triangle_3[0] * x + triangle_3[1] * y + triangle_3[2]) >=0 ): 
                        Map.occGrid[i, j]=1
                # Rectangle with clearance
                if ((upperRectangleLine_1[0] * x + upperRectangleLine_1[1] * y + upperRectangleLine_1[2]) >= -Map.robot_radius and \
                    (upperRectangleLine_2[0] * x + upperRectangleLine_2[1] * y + upperRectangleLine_2[2]) <=  Map.robot_radius and \
                    (upperRectangleLine_3[0] * x + upperRectangleLine_3[1] * y + upperRectangleLine_3[2]) >= -Map.robot_radius): 
                    Map.occGrid[i, j]=2
                    # Rectangle
                    if ((upperRectangleLine_1[0] * x + upperRectangleLine_1[1] * y + upperRectangleLine_1[2]) >= 0 and \
                    (upperRectangleLine_2[0] * x + upperRectangleLine_2[1] * y + upperRectangleLine_2[2]) <=  0 and \
                    (upperRectangleLine_3[0] * x + upperRectangleLine_3[1] * y + upperRectangleLine_3[2]) >= 0): 
                        Map.occGrid[i, j]=1
                # Rectangle with clearance
                if ((lowerRectangleLine_1[0] * x + lowerRectangleLine_1[1] * y + lowerRectangleLine_1[2]) >= -Map.robot_radius and \
                    (lowerRectangleLine_2[0] * x + lowerRectangleLine_2[1] * y + lowerRectangleLine_2[2]) <=  Map.robot_radius and \
                    (lowerRectangleLine_3[0] * x + lowerRectangleLine_3[1] * y + lowerRectangleLine_3[2]) <= Map.robot_radius): 
                        Map.occGrid[i, j]=2
                        # Rectangle
                        if ((lowerRectangleLine_1[0] * x + lowerRectangleLine_1[1] * y + lowerRectangleLine_1[2]) >= 0 and \
                        (lowerRectangleLine_2[0] * x + lowerRectangleLine_2[1] * y + lowerRectangleLine_2[2]) <=  0 and \
                        (lowerRectangleLine_3[0] * x + lowerRectangleLine_3[1] * y + lowerRectangleLine_3[2]) <= 0): 
                            Map.occGrid[i, j]=1   
               
                # hexagon with clearance
                if ((hexagonLine_1[0] * x + hexagonLine_1[1] * y + hexagonLine_1[2]) >=  -Map.robot_radius and \
                    (hexagonLine_2[0] * x + hexagonLine_2[1] * y + hexagonLine_2[2]) >=  -Map.robot_radius and \
                    (hexagonLine_3[0] * x + hexagonLine_3[1] * y + hexagonLine_3[2]) <=  Map.robot_radius  and \
                    (hexagonLine_4[0] * x + hexagonLine_4[1] * y + hexagonLine_4[2]) <=  Map.robot_radius  and \
                    (hexagonLine_5[0] * x + hexagonLine_5[1] * y + hexagonLine_5[2]) <=  Map.robot_radius and \
                    (hexagonLine_6[0] * x + hexagonLine_6[1] * y + hexagonLine_6[2]) >=  -Map.robot_radius ): 
                    Map.occGrid[i, j]=2

                    # hexagon 
                    if ((hexagonLine_1[0] * x + hexagonLine_1[1] * y + hexagonLine_1[2]) >= 0 and \
                        (hexagonLine_2[0] * x + hexagonLine_2[1] * y + hexagonLine_2[2]) >= 0 and \
                        (hexagonLine_3[0] * x + hexagonLine_3[1] * y + hexagonLine_3[2]) <= 0 and \
                        (hexagonLine_4[0] * x + hexagonLine_4[1] * y + hexagonLine_4[2]) <= 0 and \
                        (hexagonLine_5[0] * x + hexagonLine_5[1] * y + hexagonLine_5[2]) <= 0 and \
                        (hexagonLine_6[0] * x + hexagonLine_6[1] * y + hexagonLine_6[2]) >= 0 ): 
                        Map.occGrid[i, j]=1

    @classmethod
    def isValid(self, pos): 
        rows, cols = Map.occGrid.shape 
        x, y = pos
        j = x
        i = rows - 1 - y
        return Map.occGrid[i, j]==0

